# Redis backup and migration

This notebook walks through saving the state of a Redis database and restoring it on another instance by providing the target IP address and port. The workflow serialises every key using the native Redis commands for its data type (strings, hashes, lists, sets, sorted sets) while preserving TTL metadata and falling back to `DUMP`/`RESTORE` for unsupported types.


## How to use this notebook

1. Update the connection configurations (`SOURCE_REDIS_CONFIG` and `TARGET_REDIS_CONFIG`) with the IP address, port, database, and credentials for the instances you want to use.
2. Run the **Connection check** section to confirm access to the databases.
3. Execute the **Create backup** section to generate a JSON file containing the entire state of the source Redis instance.
4. Optionally transfer the file to another machine.
5. Run the **Restore backup** section to recreate the data on the target Redis instance.

⚠️ Cells that interact with Redis perform real operations on the database: enable the `FLUSH_TARGET_BEFORE_RESTORE` option carefully.


In [ ]:
from pathlib import Path
from typing import Any, Dict

from redis.exceptions import ConnectionError, ResponseError

from redis_backup import (
    build_redis_client,
    create_redis_backup,
    display_backup_summary,
    get_database_overview,
    load_backup_from_file,
    restore_redis_backup,
    save_backup_to_file,
)


In [ ]:
# Redis instance configuration
SOURCE_REDIS_CONFIG: Dict[str, Any] = {
    "host": "127.0.0.1",  # Source instance IP or hostname
    "port": 6379,
    "db": 0,
    "username": None,  # Set a value if the server requires ACL authentication
    "password": None,
}

TARGET_REDIS_CONFIG: Dict[str, Any] = {
    "host": "127.0.0.1",  # Target instance IP or hostname
    "port": 6380,
    "db": 0,
    "username": None,
    "password": None,
}

BACKUP_PATH = Path("redis_backup.json")  # Local path of the backup file
SCAN_COUNT = 1000  # Number of keys inspected per iteration during the SCAN
FLUSH_TARGET_BEFORE_RESTORE = False  # Set True to flush the target DB before restoring


In [ ]:
# Helper functions are provided by the `redis_backup` module.
# You can open the `redis_backup.py` file to review the implementation or
# reuse them in other Python scripts.


In [ ]:
# Connection check (optional but recommended)
for label, configuration in (("Source", SOURCE_REDIS_CONFIG), ("Target", TARGET_REDIS_CONFIG)):
    try:
        overview = get_database_overview(configuration)
        print(
            f"{label}: Redis {overview['redis_version']} (mode {overview['redis_mode']}) - "
            f"{overview['key_count']} keys in DB {overview['db']}"
        )
    except ConnectionError as error:
        print(f"{label}: unable to connect -> {error}")


In [ ]:
# Create the backup from the source Redis instance
try:
    backup_payload = create_redis_backup(SOURCE_REDIS_CONFIG, scan_count=SCAN_COUNT)
    save_backup_to_file(backup_payload, BACKUP_PATH)
    print(f"Backup completed: {backup_payload['metadata']['key_count']} keys saved.")
    print(f"File generated at: {BACKUP_PATH.resolve()}")
    display_backup_summary(backup_payload)
except ConnectionError as error:
    print(f"Source Redis connection failed: {error}")
except ResponseError as error:
    print(f"Error during export: {error}")


In [ ]:
# Restore the backup on the target Redis instance
try:
    backup_payload = load_backup_from_file(BACKUP_PATH)
    restored_keys = restore_redis_backup(
        backup_payload,
        TARGET_REDIS_CONFIG,
        flush_target=FLUSH_TARGET_BEFORE_RESTORE,
    )
    print(
        f"Restore completed: {restored_keys} keys imported into DB {TARGET_REDIS_CONFIG.get('db', 0)}."
    )
except FileNotFoundError:
    print(f"Backup file not found: {BACKUP_PATH}")
except ConnectionError as error:
    print(f"Target Redis connection failed: {error}")
except ResponseError as error:
    print(f"Error during restore: {error}")
